In [1]:
import numpy as np
from tqdm import tqdm

from gale_shapley import run_matching
from oneshot import oneshot

In [4]:
def dict_to_list(dictionary):
	lst = [None] * len(dictionary.keys())
	for i in range(len(lst)):
		lst[i] = dictionary.get(i)
	return lst

In [2]:
def compute_metrics(bins):
	counts = np.array([len(bin) for bin in bins])
	num_students = np.sum(counts)

	match_ratio = 1 - (counts[-1] / num_students)
	average_placement = np.sum(counts[1:] / num_students * np.array(range(1, 13)))

	return match_ratio, average_placement

In [8]:
# Compute statistics for all permutations of sel = {0, 1}, rnk = {0, 1}, adm = {1, 2, 3}
res_mr = np.zeros((2, 2, 3))
res_ap = np.zeros((2, 2, 3))

for sel in range(0, 2):
	for rnk in range(0, 2):
		for adm in range(1, 4):
			print(f'Computing metrics for sel={sel}, rnk={rnk}, adm={adm}')
			match_ratios, average_placements = [], []

			# Compute metrics over different random states
			for random_state in tqdm(range(1, 16)):
				# Generate input
				students, schools, student_info, school_info = oneshot(seed=random_state, return_list=True, sel=sel, rnk=rnk, adm=adm)
				bins, _, _ = run_matching(students, student_info, schools, school_info)
				match_ratio, average_placement = compute_metrics(dict_to_list(bins))
				match_ratios.append(match_ratio)
				average_placements.append(average_placement)

			# Average metrics and append to results
			res_mr[sel, rnk, adm-1] = np.mean(match_ratios)
			res_ap[sel, rnk, adm-1] = np.mean(average_placements)

Computing metrics for sel=0, rnk=0, adm=1


100%|██████████| 15/15 [02:08<00:00,  8.56s/it]


Computing metrics for sel=0, rnk=0, adm=2


100%|██████████| 15/15 [02:09<00:00,  8.66s/it]


Computing metrics for sel=0, rnk=0, adm=3


...

100%|██████████| 15/15 [01:51<00:00,  7.44s/it]


Computing metrics for sel=1, rnk=1, adm=2


100%|██████████| 15/15 [01:55<00:00,  7.72s/it]


Computing metrics for sel=1, rnk=1, adm=3


100%|██████████| 15/15 [01:55<00:00,  7.67s/it]


In [4]:
res_mr = np.array([
	[[0.98319345, 0.98327485, 0.98320374], [0.9683883 , 0.96833684, 0.96841076]],
	[[0.97411743, 0.97424936, 0.97401918], [0.9262793 , 0.92634105, 0.92631485]]
])

res_ap = np.array([
	[[2.16035368, 2.15952842, 2.15946573], [1.21483228, 1.21515977, 1.21434105]],
	[[1.62627649, 1.62579368, 1.62461567], [2.01865357, 2.02060538, 2.01978386]]])

In [15]:
for adm in range(3):
	print(res_mr[0, 0, adm], res_mr[0, 1, adm], res_mr[1, 0, adm], res_mr[1, 1, adm])
	print(res_ap[0, 0, adm], res_ap[0, 1, adm], res_ap[1, 0, adm], res_ap[1, 1, adm])

0.9831934502923978 0.9683883040935672 0.9741174269005847 0.9262792982456141
2.1603536842105266 1.2148322807017542 1.62627649122807 2.018653567251462
0.9832748538011695 0.968336842105263 0.9742493567251461 0.9263410526315788
2.1595284210526318 1.2151597660818714 1.6257936842105265 2.0206053801169594
0.9832037426900585 0.9684107602339183 0.9740191812865499 0.9263148538011696
2.159465730994152 1.214341052631579 1.6246156725146197 2.019783859649123


In [3]:
mr = []
ap = []

# Average over combination strategies
for random_state in range(1, 51):
	bins = dict_to_list(np.load(f'Data/Simulation/results_rs{random_state}/bins.npy', allow_pickle=True).item())
	match_ratio, average_placement = compute_metrics(bins)
	mr.append(match_ratio)
	ap.append(average_placement)

NameError: name 'dict_to_list' is not defined

In [5]:
print(np.mean(mr), np.mean(ap))

0.9542745263157894 1.60573950877193


In [6]:
match_ratios, average_placements = [], []

# 1 -> open (stage 5); 2 -> EdOpt (stage 6); 3 -> screen (stage 7)
for adm in range(1, 4):
	# Compute metrics over different random states
	for random_state in tqdm(range(1, 16)):
		# Generate input
		students, schools, student_info, school_info = oneshot(seed=random_state, return_list=True, adm=adm)
		bins, _, _ = run_matching(students, student_info, schools, school_info)
		match_ratio, average_placement = compute_metrics(dict_to_list(bins))
		match_ratios.append(match_ratio)
		average_placements.append(average_placement)

	# Average metrics and append to results
	print(f'Stage {adm+4}:', np.mean(match_ratios), np.mean(average_placements))

100%|██████████| 15/15 [02:44<00:00, 10.94s/it]


Stage 5: 0.9831934502923978 2.1603536842105266


100%|██████████| 15/15 [02:48<00:00, 11.22s/it]


Stage 6: 0.983206081871345 2.1603382456140348


100%|██████████| 15/15 [02:42<00:00, 10.83s/it]

Stage 7: 0.9832012475633528 2.1605592202729045


In [3]:
students = np.load('Data/Generated/simulation_results_rs19/student_rankings.npy', allow_pickle=True).item()
bins = np.zeros(12)

for ranking in students.values():
	bins[len(ranking) - 1] += 1

bins

array([  489.,  1068.,  2669.,  5508.,  8835., 11805., 12818., 11505.,
        8228.,  4707.,  2290.,  1328.])

In [4]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

num_students = np.sum(bins)

fig = make_subplots()
fig.add_trace(go.Bar(x=list(range(1, 13)), y=bins/num_students))

fig.update_layout(
	title='School list size',
	width=700,
	height=600,
	showlegend=False,
	xaxis=dict(title='List size', tickmode='array', tickvals=list(range(1, 13))),
	yaxis=dict(title='Percentage of students', tickformat='.0%'),
	plot_bgcolor='rgba(0, 0, 0, 0)',
	paper_bgcolor='rgba(0, 0, 0, 0)'
)

fig.show()